In [3]:
pip install folium

     |████████████████████████████████| 102 kB 2.8 MB/s ta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
from geopy.geocoders import ArcGIS
import folium

In [52]:
bike_df = pd.read_csv('bikeshare_stations.csv')
bike_df.head()

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,2541,State Capitol @ 14th & Colorado,closed,206 W. 14th St.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2021-01-04 12:00:00.000000 UTC
1,2538,Bullock Museum @ Congress & MLK,closed,1881 Congress Ave.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2021-01-04 12:00:00.000000 UTC
2,2545,ACC - Rio Grande & 12th,closed,700 W. 12th St.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,2021-01-04 12:00:00.000000 UTC
3,2546,ACC - West & 12th Street,closed,1231 West Ave.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,2021-01-04 12:00:00.000000 UTC
4,2564,5th & San Marcos,closed,991 East 5th,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,2021-01-04 12:00:00.000000 UTC


In [53]:
bike_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   station_id         102 non-null    int64  
 1   name               102 non-null    object 
 2   status             102 non-null    object 
 3   address            101 non-null    object 
 4   alternate_name     2 non-null      object 
 5   city_asset_number  77 non-null     float64
 6   property_type      84 non-null     object 
 7   number_of_docks    82 non-null     float64
 8   power_type         84 non-null     object 
 9   footprint_length   79 non-null     float64
 10  footprint_width    79 non-null     float64
 11  notes              32 non-null     object 
 12  council_district   102 non-null    int64  
 13  modified_date      102 non-null    object 
dtypes: float64(4), int64(2), object(8)
memory usage: 11.3+ KB


### Pre-processing of string data types:
- Removal of leading and trailing whitespaces
- Removal of punctuaton in address column
- Standardisation of address, for eg "St" vs "Street", "E" vs "East"
- Some entries are not actually addresses

In [61]:
# remove trailing and leading whitespaces
strip_spaces = lambda x: x.str.strip()
bike_df[['name', 'status','address','notes']] = bike_df[['name', 'status','address','notes']].apply(strip_spaces)

In [62]:
# remove punctuation in address column
bike_df['address'] = bike_df['address'].str.replace('.', '', regex=True) #not word or not space

In [ ]:
# Some addresses are invalid. Let's replace these with the station name
bike_df[bike_df['address'].str.contains('Presented')] 

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
23,2575,Riverside @ S. Lamar,active,Presented by Austin Parks Foundation,NaN,16736.0,parkland,15.0,solar,45.0,5.0,parkland at ROW/easement,5,2021-01-04 12:00:00.000000 UTC
34,2503,South Congress & James,active,Presented by The Austin Chronicle,NaN,16730.0,sidewalk,9.0,solar,26.0,7.5,on bulb-out,9,2021-01-04 12:00:00.000000 UTC
48,2549,Long Center @ South 1st & Riverside,active,Presented by Austin Energy,NaN,16750.0,sidewalk,13.0,solar,40.0,5.0,NaN,9,2021-01-04 12:00:00.000000 UTC
53,2566,Pfluger Bridge @ W 2nd Street,active,Presented by Austin Parks Foundation,NaN,16762.0,sidewalk,19.0,solar,55.0,5.0,adjacent to parkland,9,2021-01-04 12:00:00.000000 UTC
54,2499,City Hall / Lavaca & 2nd,active,Presented by Graves Dougherty Hearon & Moody,NaN,16732.0,sidewalk,19.0,solar,55.0,5.0,check dock #-18?,9,2021-01-04 12:00:00.000000 UTC
72,2501,5th & Bowie,active,Presented by Whole Foods Market,NaN,16728.0,paid_parking,15.0,solar,45.0,5.0,NaN,9,2021-01-04 12:00:00.000000 UTC


In [64]:
# Find rows where address is incorrect and replace with bike station name
bike_df.loc[bike_df['address'].str.contains('Presented'), 'address'] = bike_df['name']

In [65]:
# Fill null values in address column with bike station name
bike_df["address"] = bike_df["address"].fillna(bike_df["name"])

In [66]:
# Standardise address, for eg. "Road" is replaced by "Rd" 

words_dict = {'Street' : 'St', 'Avenue' : 'Ave', 'Road':'Rd', 
              'Drive': 'Dr', 'Boulevard' : 'Blvd', 'North' : 'N', 
              'South' : 'S', 'East' : 'E', 'West': 'W'}

# get(key, value)
bike_df['address'] = bike_df['address'].apply(lambda x: ' '.join([words_dict.get(i, i) for i in x.split()]))

## Locate longitude and latitude of the bike stations
Using ArcGIS, a web-based mapping software, we can input the given address of the bike stations from the dataset and get the full address, longtitude and latitude.  
With this information, we can create visuals to see location of bike stations and journeys taken by bike riders.  

We will create three new columns:
- full_adresss
- longitude
- latitude

Not all the addresses generated were accurate so these had to be changed manually.

In [67]:
#Add 'Austin Texas' to give more information
bike_df['address'] = bike_df['address'] + ' Austin Texas' 

In [69]:
# Initiate ArcGIS a
nom = ArcGIS()

In [70]:
# Create column that gets address, latitude and longitude
bike_df['coordinates']=bike_df['address'].apply(nom.geocode)

#extract coordinates and store it in lat and lon columns
bike_df['full_address']=bike_df['coordinates'].apply(lambda x:x.address)
bike_df['lat']=bike_df['coordinates'].apply(lambda x:x.latitude)
bike_df['lon']=bike_df['coordinates'].apply(lambda x:x.longitude)

In [116]:
# Not all rows were successfully matched, find rows where location was not found to be Austin and change manually
# station_id 2546, 2566, 2537 are incorrect.

bike_df[~bike_df['full_address'].str.contains('Austin')] 

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date,coordinates,full_address,lat,lon
3,2546,ACC - West & 12th Street,closed,1231 W Ave Austin Texas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,2021-01-04 12:00:00.000000 UTC,"(1231 S Ave A St, Bellville, Texas, 77418, (29...","1231 S Ave A St, Bellville, Texas, 77418",29.936565,-96.249771
39,2550,4th/Guadalupe @ Republic Square,active,4th/Guadalupe @ Republic Square Austin Texas,NaN,NaN,sidewalk,17.0,solar,30.0,5.0,NaN,9,2022-03-04 10:28:00.000000 UTC,"(Republic Square Park, (30.267720000000054, -9...",Republic Square Park,30.267720,-97.746820
43,3455,Republic Square @ 5th & Guadalupe,closed,Federal Courthouse Plaza Austin Texas,NaN,16741.0,sidewalk,13.0,non-metered,40.0,5.0,Moved back to 4th/Guadalupe,9,2022-03-04 10:24:00.000000 UTC,"(Federal Courthouse, (30.268820000000062, -97....",Federal Courthouse,30.268820,-97.744170
53,2566,Pfluger Bridge @ W 2nd Street,active,Pfluger Bridge @ W 2nd St Austin Texas,NaN,16762.0,sidewalk,19.0,solar,55.0,5.0,adjacent to parkland,9,2021-01-04 12:00:00.000000 UTC,"(Pfluger St W & S 2nd St, Pflugerville, Texas,...","Pfluger St W & S 2nd St, Pflugerville, Texas, ...",30.438780,-97.622870
91,2537,West & 6th St.,active,603 W Ave Austin Texas,NaN,16737.0,undetermined_parking,13.0,solar,40.0,5.0,NaN,9,2021-01-04 12:00:00.000000 UTC,"(603 S Ave A St, Bellville, Texas, 77418, (29....","603 S Ave A St, Bellville, Texas, 77418",29.938709,-96.248655


In [123]:
# Manually change data for station_id 2546, 2566, 2537 based on long and lat on Google Maps
bike_df.loc[bike_df['station_id'] == 2546, ['full_address', 'lat', 'lon']]  = ['1231 W Ave, Austin, Texas, 78701', 30.276460, -97.748220]

bike_df.loc[bike_df['station_id'] == 2566, ['full_address', 'lat', 'lon']]  = ['W 2nd St, Austin, Texas, 78701', 30.264320, -97.744490]

bike_df.loc[bike_df['station_id'] == 2537, ['full_address', 'lat', 'lon']]  = ['603 W Ave, Austin, Texas, 78701', 30.270678, -97.750389]

In [125]:
bike_df.columns

# need to drop irrelavant columns

Index(['station_id', 'name', 'status', 'address', 'alternate_name',
       'city_asset_number', 'property_type', 'number_of_docks', 'power_type',
       'footprint_length', 'footprint_width', 'notes', 'council_district',
       'modified_date', 'coordinates', 'full_address', 'lat', 'lon'],
      dtype='object')

## Bike station locations
Now we can plot the locations of all the bike stations on a map

In [162]:
map_new=folium.Map(width=1200, height=400,zoom_start=20)
list_coor=bike_df[['full_address','lat','lon']].values.tolist()
for i in list_coor:
    map_new.add_child(folium.Marker(location=[i[1],i[2]],
                                popup=i[0],icon=folium.Icon(color='green')))
    
sw = bike_df[['lat', 'lon']].min().values.tolist()
ne = bike_df[['lat', 'lon']].max().values.tolist()

map_new.fit_bounds([sw, ne]) 

map_new
# map_new.save('bike station.html')

In [22]:
bike_df.to_csv('bike_station_cleaned.csv')